In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from functools import cmp_to_key

# importing libraries and creating aliases

In [2]:
startdate = '2023-01-01'
enddate = '2023-10-31'

ticker_file = 'Tickers_Example.csv' 
tickers = pd.read_csv(ticker_file, header=None).values.flatten().tolist() 

# storing exchange rates
CAD_transfer = yf.Ticker('CADUSD=x')
transferRate = CAD_transfer.history(start=startdate, end=enddate)
CAD_to_USD = transferRate[['Close']]
CAD_to_USD.index = CAD_to_USD.index.strftime('%Y-%m-%d')
    
def currency_exchange(base, currency):
    ''' 
    currency_exchange(base, currency) expects a dataframe with close prices & string corresponding 
    to a currency type, and, if necessary, returns & converts to CAD using historical exchange rates
    '''
    base.index = base.index.strftime('%Y-%m-%d')
    if currency == 'USD':
        exchange_rates = pd.DataFrame()
        exchange_rates['Close'] = base['Close']/CAD_to_USD['Close']
        exchange_rates = exchange_rates.dropna()
        return exchange_rates
    else:
        return base

In [3]:
def is_valid_currency(ticker):
    '''
    is_valid_currency(ticker) expects a string corresponding to an asset ticker & returns TRUE if it is 
    denominated in CAD or USD, and FALSE else
    '''
    stock_info = yf.Ticker(ticker).info
    currency = stock_info.get('currency', '').upper() # retrieving the stock currency
    return currency == 'USD' or currency == 'CAD' # returns T/F based on currency

In [4]:
def get_stock_data(ticker):
    '''
    get_stock_data(ticker) expects a string corresponding to an asset ticker and returns a dictionary, if the ticker
    is valid and denominated in CAD/USD, containing the ticker name, stock info, historical close price, and 
    historical daily volume
    '''
    # get stock info
    stock_info = yf.Ticker(ticker).info
    
    # check if the currency is valid
    if is_valid_currency(ticker):
        # download historical data
        data = yf.download(ticker, start=startdate, end=enddate)
        
        # returning daily volume
        monthly_volume = data['Volume'].resample('D').sum()
        
        #turning American stocks to CAD$
        stock_price = currency_exchange(data[['Close']], stock_info['currency'])
        
        # create a dictionary for the ticker
        result = {
            'ticker': ticker,
            'info': stock_info,
            'close': stock_price,
            'volume': pd.DataFrame(monthly_volume)
        }
        return result
    else:
        print(f"Skipping {ticker}: Invalid currency.")
        return None

# accumulates a list of dictionaries through get_stock_data    
def process_tickers(ticker_list):
    '''
    process_tickers(ticker_list) expects a list of tickers, and returns a list of dictionaries with each ticker's
    ticker name, info, close price, and daily volume (for valid entries & those denominated in CAD/USD)
    '''
    result_list = []
    for ticker in ticker_list:
        stock_data = get_stock_data(ticker)
        if stock_data is not None:
            result_list.append(stock_data)
    return result_list

result_list = process_tickers(tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Skipping AGN: Invalid currency.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Skipping CELG: Invalid currency.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Skipping MON: Invalid currency.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Skipping RTN: Invalid currency.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
def filter_stocks(result_list, min_avg_volume=150000, min_trading_days=18):
    '''
    filter_stocks(result_list, min_avg_volume, min_trading_days) expects a list of dictionaries with info from
    from get_stock_data(tickers) and filters to only contain dictionaries for assets with a mininum monthly avg 
    volume of min_avg_volume (only including months with a minimum of min_trading_days trading days)
    '''
    filtered_list = []

    for stock_data in result_list:
        ticker = stock_data['ticker']
        volume_data = stock_data['volume']

        # calculate average monthly volume
        monthly_avg_volume = {}
        for date, volume in volume_data.iterrows():
            month = date.strftime('%Y-%m')
            if month not in monthly_avg_volume:
                monthly_avg_volume[month] = {'total_volume': 0, 'days_counted': 0}
            if volume['Volume'] > 0 or monthly_avg_volume[month]['days_counted'] < min_trading_days:
                monthly_avg_volume[month]['total_volume'] += volume['Volume']
                monthly_avg_volume[month]['days_counted'] += 1

        # remove stocks with less than min_trading_days in any month or average volume less than min_avg_volume
        valid_months = [month for month, data in monthly_avg_volume.items() if data['days_counted'] >= min_trading_days]
        avg_monthly_volume = sum(monthly_avg_volume[month]['total_volume'] / data['days_counted'] for month, data in monthly_avg_volume.items() if data['days_counted'] > 0) / len(valid_months) if valid_months else 0

        if avg_monthly_volume >= min_avg_volume:
            filtered_list.append(stock_data)

    return filtered_list

filtered_list = filter_stocks(result_list)

In [6]:
def return_industry(tickers):
    '''
    return_industry(tickers) expects a list of dictionaries and returns a list of all the ticker's industries
    '''
    industries = []
    for i in range (len (tickers)):
        stock = tickers[i]
        sector = stock["info"]['sector']
        industries.append(sector)
    return industries

list_of_industries = return_industry(filtered_list)

def industry_count(industry_list):
    '''
    industry_count(industry_list) takes a list of strings corresponding to industry names and returns a dictionary
    with each industry and the number of times it occurs in said list
    '''
    industry_dict={}
    while len(industry_list) > 0:
        industry_dict[industry_list[0]] = industry_list.count(industry_list[0])
        industry_list = remove_items(industry_list, industry_list[0])
    return industry_dict

def remove_items(lst, item): 
    '''
    remove_items(lst, item) takes a list of strings and a string and removes all occurances of said string in lst
    '''
    resulting_list = [i for i in lst if i != item] 
    return resulting_list

count_of_industries = industry_count(list_of_industries)

#Sorts the dictionary of industries by descending number of stocks in the industry
sorted_dict_of_industries = sorted(count_of_industries.items(),key=lambda x:x[1], reverse = True)

In [7]:
#creating a dictionary of stocks within the given industry
def stocks_in_industry(industry_dict):
    '''
    stocks_in_industry(industry_dict) expects a dictionary with sectors and their occurance counts, and 
    returns a dictionary where each key represents a sector and its value is a list of stock info belonging to 
    that sector
    '''
    industry_portfolio = {}
    for i in range (len (industry_dict)):
        industry_portfolio[(industry_dict[i][0])] = stock_grouper(filtered_list, (industry_dict[i][0]))
    return industry_portfolio

def stock_grouper(stock_list, industry):
    '''
    stock_grouper(stock_list, industry) expects a list of stock dictionaries and a string corresponding to a sector
    and returns a list of stock dictionaries in said sector
    '''
    stocks = []
    for i in range (len (stock_list)):
        stock = stock_list[i]
        sector = stock["info"]['sector']
        if sector == industry:
            stocks.append(stock)
    return stocks
            
stocks_in_industry_list = stocks_in_industry(sorted_dict_of_industries)      

In [8]:
def calculate_beta(market_data, stock_dict):
    '''
    calculate_beta(market_data, stock_dict) calculates and returns the beta coefficient for a group of 
    stocks (stock_dict) relative to a market index (market_data -- we use the S&P500 later on)
    '''
    # calculate market index returns
    returns = market_data
    returns = returns.rename(columns={'Close': 'Market'})
    returns = returns.pct_change()
    returns.drop(index=returns.index[0], inplace=True)
    betas = []
    
    # calculate returns for the stock and adds it to the overall dataframe
    stockReturns = stock_dict['close']
    stockReturns = stockReturns.rename(columns={'Close': stock_dict['ticker']})
    stockReturns = stockReturns.pct_change()
    stockReturns.drop(index=stockReturns.index[0], inplace=True)
    returns = returns.merge(stockReturns,left_index=True,right_index=True)
    
    # calculates covariance and market variance
    returnsCov = returns.cov()
    var = returns['Market'].var()
    beta = returnsCov/var
    
    return beta.iat[1,0]

In [9]:
# aggregate score for each industry is initially zero
industry_scores = {}
for sector in stocks_in_industry_list:
    industry_scores[sector] = 0
    
# get market data (we are using s&p 500)
market_data = yf.Ticker('^GSPC').history(start=startdate, end=enddate)[['Close']]#,period='1y',interval='1d'
market_data.index = market_data.index.strftime('%Y-%m-%d')

# consider beta for each industry (relative to s&p 500)
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    beta_avg = 0
    for stock in stock_lst:
        ourbeta = calculate_beta(market_data,stock)
        beta_avg += ourbeta
    beta_avg /= len(stock_lst)
    industry_scores[sector] += beta_avg*0.3

In [10]:
# consider standard deviations for each industry
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    std_avg = 0
    for stock in stock_lst:
        close_pct = stock['close'].pct_change()
        close_pct.drop(index=close_pct.index[0], inplace=True)
        std_avg += close_pct['Close'].std()
    std_avg /= len(stock_lst)
    industry_scores[sector] += std_avg*20

The following code will compare the average 52 week high and low of each industry against each other. Our reasoning behind this statistic was to look at the previous volatility of the stock. If the resulting number was high, it would indicate a large potiential for movement and would signal a risky investment. If the resulting number was low, it would indicate a low potiential for movement and would signal a relatively stable stock that we do not want for our final portfolio.

In [11]:
# consider 52 week range (high divided by low) for each industry
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    hilo_avg = 0
    for stock in stock_lst:
        hilo_avg += stock['info']['fiftyTwoWeekHigh']/stock['info']['fiftyTwoWeekLow']
    hilo_avg /= len(stock_lst)
    industry_scores[sector] += hilo_avg*0.1

Earlier in the semester, we proved that diversification lowered the standard deviation of the portfolio's daily returns, directly lowering the riskiness of our portfolio. We decided to negate the effect of this by grouping stocks by their industries, and by doing so we increase the correlation of our final portfolio as we add the entire industry in as opposed to singular stocks. We also want to buy the least amount of stocks possible, as that would further decrease our diversifiction.

This is why we decided to alter the final aggregate risk score of each industry by the number of stocks in the portfolio. If we had decided to not do this, industries that possess only a few volatile stocks would form the majority of our portfolio, which would create a portfolio with neutral correlation. This would make our strategy virtually worthless as there was a high likelihood that the stocks would diversify the risk through multiple industries. By ensuring a high correlation, each stock would be moving in the same direction, which would lower the risk of diversifiction.

In [12]:
# Adjust industry score by number of stocks within
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    industry_scores[sector] *= 1-2**(-len(stock_lst))

In [13]:
# Format and print final result

# convert industry_scores to a list so we can sort it
industry_scores_temp = []
for sector in industry_scores:
    industry_scores_temp.append((sector,industry_scores[sector]))
industry_scores = industry_scores_temp

# comparison function that sorts tuples by their second value in descending order
def sort_by_second_desc(a, b):
    '''
    sort_by_second_desc(a, b) expects tow tuples to sort them by their second value in descending order. It 
    returns 1 if a[1] < b[1], -1 if b[1] < a[1], and 0 if they are equal
    '''
    if a[1] < b[1]:
        return 1
    elif a[1] > b[1]:
        return -1
    else:
        return 0
    
#sort industries by aggregate score in descending order
industry_scores.sort(key=cmp_to_key(sort_by_second_desc))

In [14]:
# Plan: consider the most volatile industries first and assign each stock in those industries the highest weighting allowed
N_stocks = 10
principal = 750000

# stock_weightings[i] is the weight of the ith stock considered
stock_weightings = [0.2,0.2,0.2,0.1,0.05,0.05,0.05,0.05,0.05,0.05]

# The top 3 stocks each receive a weighting of 20% of the total portfolio value.
# The next stock receives a weighting of 10%.
# The remaining 6 stocks each receive a weighting of 5%.

# a list of tuples, each tuple stores a ticker string and its corresponding portfolio weighting
stock_percentages = []
for industry in industry_scores:
    
    if len(stock_percentages) >= N_stocks:
        break

    industry_name = industry[0]
    print(industry_name)
    industry_score = industry[1]
    stock_lst = stocks_in_industry_list[industry_name]
    
    # as a tie breaker, we first consider the stocks within the current industry with the highest std
    stock_stds = []
    for stock in stock_lst:
        stock_returns = stock['close'].pct_change()
        stock_returns.drop(index=stock_returns.index[0], inplace=True)
        stock_stds.append((stock['ticker'],stock_returns['Close'].std()))
    stock_stds.sort(key=cmp_to_key(sort_by_second_desc))
    print(stock_stds)
    for stock in stock_stds:
        if len(stock_percentages) >= N_stocks:
            break
        stock_percentages.append((stock[0],stock_weightings[len(stock_percentages)]))

print(stock_percentages)

Portfolio_Final = {'Ticker': [],
                   'Price': [],
                   'Currency': [],
                   'Shares': [],
                   'Value': [],
                   'Weight': []
                  }

curr_cad_usd = yf.Ticker('CADUSD=x').info['previousClose']
for stock in stock_percentages:
    stock_info = yf.Ticker(stock[0]).info
    curr_price = stock_info['currentPrice']
    Portfolio_Final['Ticker'].append(stock[0])
    Portfolio_Final['Price'].append(curr_price)
    Portfolio_Final['Currency'].append(stock_info['currency'])
    num_shares = 0
    if stock_info['currency'] == 'USD':
        num_shares = principal*stock[1]/(curr_price/curr_cad_usd)
    else:
        num_shares = principal*stock[1]/curr_price
    Portfolio_Final['Shares'].append(num_shares)
    Portfolio_Final['Value'].append(principal*stock[1])
    Portfolio_Final['Weight'].append(stock[1])
    
Portfolio_Final = pd.DataFrame(data=Portfolio_Final,index=[(i+1) for i in range(N_stocks)])

Technology
[('SHOP.TO', 0.03525100825025024), ('QCOM', 0.02069628686865636), ('TXN', 0.015591193442479338), ('ACN', 0.014876924598013272), ('AAPL', 0.013269600515218563)]
Financial Services
[('USB', 0.025563254659290872), ('PYPL', 0.02270225473418308), ('AIG', 0.01768686150875919), ('AXP', 0.017153095145726875), ('BAC', 0.016648149352155615), ('BK', 0.01624253580354934), ('C', 0.016224530521288704), ('BLK', 0.013482523539544603), ('TD.TO', 0.010949856846754842), ('RY.TO', 0.00926191049681605)]
[('SHOP.TO', 0.2), ('QCOM', 0.2), ('TXN', 0.2), ('ACN', 0.1), ('AAPL', 0.05), ('USB', 0.05), ('PYPL', 0.05), ('AIG', 0.05), ('AXP', 0.05), ('BAC', 0.05)]


In [15]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight
1,SHOP.TO,80.66,CAD,1859.657823,150000.0,0.20
2,QCOM,204.05,USD,537.458292,150000.0,0.20
3,TXN,195.01,USD,562.373030,150000.0,0.20
4,ACN,282.29,USD,194.247697,75000.0,0.10
5,AAPL,192.25,USD,142.611657,37500.0,0.05
6,USB,40.55,USD,676.130484,37500.0,0.05
7,PYPL,62.99,USD,435.261012,37500.0,0.05
8,AIG,78.82,USD,347.844343,37500.0,0.05
9,AXP,240.00,USD,114.237880,37500.0,0.05
10,BAC,39.99,USD,685.598678,37500.0,0.05
